In [2]:
import os
import numpy as np
import librosa
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import optuna

# ------------------- Load SAVEE -------------------
def load_savee_audio(dataset_path):
    audio_files = []
    labels = []

    label_map = {
        'a': 'angry', 'd': 'disgust', 'f': 'fear', 'h': 'happy',
        'n': 'neutral', 'sa': 'sad', 'su': 'surprise'
    }

    for file in os.listdir(dataset_path):
        if file.endswith(".wav"):
            parts = file.split('_')
            emotion_code = parts[1][:2] if parts[1][:2] in label_map else parts[1][0]
            if emotion_code in label_map:
                emotion = label_map[emotion_code]
                audio_files.append(os.path.join(dataset_path, file))
                labels.append(emotion)

    return audio_files, labels

# Feature extraction: MFCC + ZCR + RMS
def extract_custom_features(file_path, sr=22050):
    y, sr = librosa.load(file_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    zcr = librosa.feature.zero_crossing_rate(y)
    rms = librosa.feature.rms(y=y)
    features = np.concatenate((mfcc, zcr, rms), axis=0)
    return np.mean(features.T, axis=0)

dataset_path = "C:/Users/samhi/OneDrive/문서/College/s6/Speech Processing/Endsem/archive/ALL"

audio_files, labels = load_savee_audio(dataset_path)

X, y_clean = [], []
for file, label in tqdm(zip(audio_files, labels), total=len(audio_files)):
    try:
        features = extract_custom_features(file)
        X.append(features)
        y_clean.append(label)
    except Exception as e:
        print(f"Error processing {file}: {e}")

X = np.array(X)
y = LabelEncoder().fit_transform(y_clean)
X = StandardScaler().fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ------------------- DCNN with Optuna -------------------
def objective(trial):
    model = Sequential([
        Conv1D(trial.suggest_int('filters1', 32, 128), kernel_size=trial.suggest_int('kernel1', 3, 7),
               activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Dropout(trial.suggest_float('dropout', 0.2, 0.5)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(trial.suggest_float('dropout', 0.2, 0.5)),
        Dense(len(set(y)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, epochs=20, batch_size=trial.suggest_int('batch', 16, 64), verbose=0,
              validation_data=(X_test.reshape(-1, X_test.shape[1], 1), y_test), callbacks=[EarlyStopping(patience=3)])
    _, acc = model.evaluate(X_test.reshape(-1, X_test.shape[1], 1), y_test, verbose=0)
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

params = study.best_params
model = Sequential([
    Conv1D(params['filters1'], kernel_size=params['kernel1'], activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(params['dropout']),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(params['dropout']),
    Dense(len(set(y)), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, epochs=50, batch_size=params['batch'], validation_data=(X_test.reshape(-1, X_test.shape[1], 1), y_test), callbacks=[EarlyStopping(patience=5)])

# Evaluation
y_pred = model.predict(X_test.reshape(-1, X_test.shape[1], 1))
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))


100%|██████████| 480/480 [01:07<00:00,  7.14it/s]
[I 2025-04-06 12:37:20,400] A new study created in memory with name: no-name-5b1c1f78-5896-4ee9-ba0d-e453448088ea


[I 2025-04-06 12:37:37,422] Trial 0 finished with value: 0.6770833134651184 and parameters: {'filters1': 69, 'kernel1': 3, 'dropout': 0.2881315839456688, 'batch': 22}. Best is trial 0 with value: 0.6770833134651184.
[I 2025-04-06 12:37:48,831] Trial 1 finished with value: 0.6979166865348816 and parameters: {'filters1': 43, 'kernel1': 4, 'dropout': 0.22322288299139398, 'batch': 38}. Best is trial 1 with value: 0.6979166865348816.
[I 2025-04-06 12:38:00,818] Trial 2 finished with value: 0.6979166865348816 and parameters: {'filters1': 126, 'kernel1': 7, 'dropout': 0.3818525408167189, 'batch': 62}. Best is trial 1 with value: 0.6979166865348816.
[I 2025-04-06 12:38:12,589] Trial 3 finished with value: 0.7291666865348816 and parameters: {'filters1': 126, 'kernel1': 6, 'dropout': 0.24222272653242335, 'batch': 60}. Best is trial 3 with value: 0.7291666865348816.
[I 2025-04-06 12:38:23,180] Trial 4 finished with value: 0.7083333134651184 and parameters: {'filters1': 53, 'kernel1': 5, 'dropout'

Epoch 1/50
8/8 [==============================] - 6s 229ms/step - loss: 1.7864 - accuracy: 0.3151 - val_loss: 1.4242 - val_accuracy: 0.4896
Epoch 2/50
8/8 [==============================] - 0s 55ms/step - loss: 1.3945 - accuracy: 0.4740 - val_loss: 1.2258 - val_accuracy: 0.5729
Epoch 3/50
8/8 [==============================] - 0s 67ms/step - loss: 1.2091 - accuracy: 0.5417 - val_loss: 1.0412 - val_accuracy: 0.6146
Epoch 4/50
8/8 [==============================] - 0s 57ms/step - loss: 1.0585 - accuracy: 0.6120 - val_loss: 1.0064 - val_accuracy: 0.6354
Epoch 5/50
8/8 [==============================] - 0s 65ms/step - loss: 0.9821 - accuracy: 0.6484 - val_loss: 0.9287 - val_accuracy: 0.6354
Epoch 6/50
8/8 [==============================] - 0s 60ms/step - loss: 0.8906 - accuracy: 0.6641 - val_loss: 0.8790 - val_accuracy: 0.6979
Epoch 7/50
8/8 [==============================] - 0s 65ms/step - loss: 0.8535 - accuracy: 0.6771 - val_loss: 0.8444 - val_accuracy: 0.6979
Epoch 8/50
8/8 [==========